# 📕 FILE 3-A: TRANSFER LEARNING & MIXED PRECISION

**Phần:** ADVANCED & PROFESSIONAL (Production-Ready)

**Mục tiêu:**
- ✅ Hiểu và áp dụng Transfer Learning
- ✅ Sử dụng pre-trained models (MobileNet, ResNet)
- ✅ Fine-tuning hiệu quả
- ✅ Mixed Precision Training để tăng tốc
- ✅ Tối ưu performance cho production

**Thời lượng:** 3-4 tuần

---

## 📚 Mục Lục

### PHẦN 1: TRANSFER LEARNING
1. Transfer Learning là gì?
2. Pre-trained Models trong TensorFlow
3. Feature Extraction
4. Fine-tuning
5. Transfer Learning với MobileNetV2
6. Transfer Learning với ResNet50
7. Best Practices

### PHẦN 2: MIXED PRECISION TRAINING
1. Mixed Precision là gì?
2. Tại sao dùng Mixed Precision?
3. Cấu hình Mixed Precision
4. Training với Mixed Precision
5. So sánh Performance
6. Best Practices

---

In [ ]:
# Import các thư viện cần thiết
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import (
    MobileNetV2, ResNet50, EfficientNetB0, VGG16
)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(f"✅ Built with CUDA: {tf.test.is_built_with_cuda()}")

---

# PHẦN 1: TRANSFER LEARNING

## 1.1 Transfer Learning là gì?

### Định nghĩa

**Transfer Learning** = Sử dụng lại kiến thức từ model đã được train trên dataset lớn

### Ví dụ thực tế

Giống như một người đã học vẽ người, sẽ dễ dàng học vẽ động vật hơn người mới bắt đầu!

### Tại sao dùng Transfer Learning?

| Vấn đề | Giải pháp với Transfer Learning |
|--------|----------------------------------|
| 📊 Ít dữ liệu | Dùng model đã học từ dataset lớn (ImageNet) |
| ⏱️ Tốn thời gian | Không cần train từ đầu |
| 💰 Tốn chi phí | Không cần GPU mạnh để train lâu |
| 🎯 Hiệu quả cao | Đạt accuracy cao với ít dữ liệu |

### Khi nào dùng Transfer Learning?

✅ **NÊN DÙNG khi:**
- Dataset nhỏ (< 10,000 images)
- Bài toán tương tự với ImageNet (phân loại ảnh)
- Cần kết quả nhanh
- Ít tài nguyên (GPU, thời gian)

❌ **KHÔNG NÊN DÙNG khi:**
- Dataset rất lớn và khác biệt (medical images, satellite...)
- Có đủ tài nguyên để train từ đầu
- Bài toán hoàn toàn mới (không giống ImageNet)

### Hai chiến lược chính

#### 1. Feature Extraction
- **Freeze** toàn bộ pre-trained layers
- Chỉ train classifier mới
- Nhanh, ít data

#### 2. Fine-tuning
- **Unfreeze** một số layers cuối
- Train lại với learning rate nhỏ
- Chậm hơn nhưng chính xác hơn

## 1.2 Pre-trained Models trong TensorFlow

### ImageNet Dataset

- 14 triệu ảnh
- 1000 classes (dog, cat, car, plane...)
- Benchmark cho Computer Vision

### Các Pre-trained Models phổ biến

| Model | Parameters | Top-1 Accuracy | Khi nào dùng |
|-------|-----------|----------------|---------------|
| **MobileNetV2** | 3.5M | 71.8% | Mobile, edge devices, tốc độ |
| **ResNet50** | 25.6M | 75.0% | Cân bằng accuracy/speed |
| **EfficientNetB0** | 5.3M | 77.1% | Hiệu quả nhất (accuracy/size) |
| **VGG16** | 138M | 71.3% | Đơn giản, dễ hiểu |
| **InceptionV3** | 23.9M | 77.9% | Accuracy cao |

### Khuyến nghị

- 🚀 **Production/Mobile**: MobileNetV2, EfficientNetB0
- 🎯 **Accuracy ưu tiên**: ResNet50, InceptionV3
- 📚 **Học tập**: MobileNetV2 (nhỏ, nhanh)
- 💻 **Research**: EfficientNet series

In [ ]:
# Load pre-trained models để so sánh

print("📦 Loading pre-trained models...\n")

models_info = []

# MobileNetV2
mobilenet = MobileNetV2(weights='imagenet', include_top=False)
models_info.append({
    'name': 'MobileNetV2',
    'params': mobilenet.count_params(),
    'model': mobilenet
})

# ResNet50
resnet = ResNet50(weights='imagenet', include_top=False)
models_info.append({
    'name': 'ResNet50',
    'params': resnet.count_params(),
    'model': resnet
})

# EfficientNetB0
efficientnet = EfficientNetB0(weights='imagenet', include_top=False)
models_info.append({
    'name': 'EfficientNetB0',
    'params': efficientnet.count_params(),
    'model': efficientnet
})

# So sánh
print("📊 SO SÁNH CÁC PRE-TRAINED MODELS:")
print("=" * 60)
print(f"{'Model':<20} {'Parameters':<15} {'Size (MB)':<12}")
print("=" * 60)

for info in models_info:
    params = info['params']
    size_mb = params * 4 / (1024 * 1024)  # Float32 = 4 bytes
    print(f"{info['name']:<20} {params:>12,}   {size_mb:>8.2f} MB")

print("=" * 60)
print("\n💡 Giải thích:")
print("   - include_top=False: Loại bỏ classifier layer (1000 classes)")
print("   - weights='imagenet': Load weights đã train trên ImageNet")
print("   - Chúng ta sẽ thêm classifier mới cho bài toán của mình")

## 1.3 Chuẩn bị Dataset: Cats vs Dogs

Chúng ta sẽ dùng dataset **Cats vs Dogs** để demo Transfer Learning.

### Dataset
- 2000 ảnh training (1000 cats, 1000 dogs)
- 1000 ảnh validation (500 cats, 500 dogs)
- Subset từ Kaggle Cats vs Dogs

In [ ]:
# Download dataset
import urllib.request
import zipfile
import os

# Download Cats vs Dogs subset
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
dataset_path = 'cats_and_dogs_filtered.zip'

if not os.path.exists('cats_and_dogs_filtered'):
    print("📥 Downloading dataset...")
    urllib.request.urlretrieve(url, dataset_path)
    
    print("📦 Extracting...")
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall('.')
    
    print("✅ Dataset ready!")
else:
    print("✅ Dataset already exists!")

# Dataset paths
base_dir = 'cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Count images
train_cats = len(os.listdir(os.path.join(train_dir, 'cats')))
train_dogs = len(os.listdir(os.path.join(train_dir, 'dogs')))
val_cats = len(os.listdir(os.path.join(validation_dir, 'cats')))
val_dogs = len(os.listdir(os.path.join(validation_dir, 'dogs')))

print("\n📊 DATASET INFO:")
print("=" * 50)
print(f"Training:")
print(f"  - Cats: {train_cats}")
print(f"  - Dogs: {train_dogs}")
print(f"  - Total: {train_cats + train_dogs}")
print()
print(f"Validation:")
print(f"  - Cats: {val_cats}")
print(f"  - Dogs: {val_dogs}")
print(f"  - Total: {val_cats + val_dogs}")
print("=" * 50)

In [ ]:
# Visualize một số ảnh
from tensorflow.keras.preprocessing import image

# Get sample images
cat_files = [os.path.join(train_dir, 'cats', f) 
             for f in os.listdir(os.path.join(train_dir, 'cats'))[:4]]
dog_files = [os.path.join(train_dir, 'dogs', f) 
             for f in os.listdir(os.path.join(train_dir, 'dogs'))[:4]]

# Plot
plt.figure(figsize=(12, 6))

for i, img_path in enumerate(cat_files + dog_files):
    img = image.load_img(img_path, target_size=(150, 150))
    plt.subplot(2, 4, i + 1)
    plt.imshow(img)
    plt.title('Cat' if i < 4 else 'Dog')
    plt.axis('off')

plt.tight_layout()
plt.show()

## 1.4 Data Pipeline với tf.keras.preprocessing

### ImageDataGenerator

Tạo data pipeline với augmentation:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image size cho MobileNetV2
IMG_SIZE = 224  # MobileNetV2 input size
BATCH_SIZE = 32

# Data augmentation cho training
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize về [0, 1]
    rotation_range=20,           # Rotate ±20 độ
    width_shift_range=0.2,       # Shift ngang 20%
    height_shift_range=0.2,      # Shift dọc 20%
    horizontal_flip=True,        # Flip ngang
    zoom_range=0.2,              # Zoom in/out 20%
    fill_mode='nearest'          # Fill pixels khi transform
)

# Chỉ rescale cho validation (KHÔNG augment)
val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # Binary classification
)

val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

print("✅ Data generators created!")
print(f"   Classes: {train_generator.class_indices}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Image size: {IMG_SIZE}x{IMG_SIZE}")

In [ ]:
# Visualize augmented images
sample_training_images, _ = next(train_generator)

plt.figure(figsize=(12, 6))
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(sample_training_images[i])
    plt.axis('off')
plt.suptitle('Augmented Training Images', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("📊 Data augmentation giúp:")
print("   ✅ Tăng số lượng data ảo")
print("   ✅ Model robust hơn với các biến thể")
print("   ✅ Giảm overfitting")

## 1.5 Feature Extraction với MobileNetV2

### Chiến lược 1: Feature Extraction

```
Pre-trained MobileNetV2 (FROZEN)
        ↓
Global Average Pooling
        ↓
Dense (NEW, TRAINABLE)
```

In [ ]:
# Load pre-trained MobileNetV2
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,  # Loại bỏ classification head
    weights='imagenet'  # Weights từ ImageNet
)

# FREEZE base model (QUAN TRỌNG!)
base_model.trainable = False

print("📦 MobileNetV2 base model:")
print(f"   Total layers: {len(base_model.layers)}")
print(f"   Trainable: {base_model.trainable}")
print(f"   Parameters: {base_model.count_params():,}")

In [ ]:
# Xây dựng model với Feature Extraction

# Input
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Pre-trained base (frozen)
x = base_model(inputs, training=False)  # training=False để không update batch norm

# Global Average Pooling
x = layers.GlobalAveragePooling2D()(x)

# Dropout để tránh overfitting
x = layers.Dropout(0.2)(x)

# Output layer
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create model
model = keras.Model(inputs, outputs)

print("\n✅ Feature Extraction Model:")
model.summary()

# Giải thích:
print("\n📚 Giải thích:")
print("   1. Base model (MobileNetV2): FROZEN → Không train")
print("   2. GlobalAveragePooling2D: Chuyển feature maps thành vector")
print("   3. Dropout(0.2): Tắt 20% neurons → Tránh overfitting")
print("   4. Dense(1, sigmoid): Binary classification")

In [ ]:
# Kiểm tra trainable parameters
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
non_trainable_params = sum([tf.size(w).numpy() for w in model.non_trainable_weights])
total_params = model.count_params()

print("📊 PARAMETERS BREAKDOWN:")
print("=" * 60)
print(f"Total parameters:        {total_params:>15,}")
print(f"Trainable parameters:    {trainable_params:>15,}  ({trainable_params/total_params*100:.2f}%)")
print(f"Non-trainable parameters:{non_trainable_params:>15,}  ({non_trainable_params/total_params*100:.2f}%)")
print("=" * 60)

print("\n💡 Chú ý:")
print("   - Chỉ train ~0.1% parameters!")
print("   - Training rất nhanh")
print("   - Phù hợp khi có ít dữ liệu")

In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("✅ Model compiled!")
print("   Optimizer: Adam (lr=0.001)")
print("   Loss: binary_crossentropy")
print("   Metrics: accuracy")

In [ ]:
# Callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_feature_extraction_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print("✅ Callbacks configured:")
print("   1. EarlyStopping: Stop nếu không cải thiện sau 5 epochs")
print("   2. ModelCheckpoint: Lưu model tốt nhất")
print("   3. ReduceLROnPlateau: Giảm LR nếu stuck")

In [ ]:
# Training
print("🚀 Training Feature Extraction Model...\n")

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training completed!")

In [ ]:
# Visualize training history
def plot_history(history, title_prefix=''):
    """Plot training history"""
    plt.figure(figsize=(14, 5))
    
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    plt.xlabel('Epoch', fontsize=11)
    plt.ylabel('Accuracy', fontsize=11)
    plt.title(f'{title_prefix}Model Accuracy', fontsize=12, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation', linewidth=2)
    plt.xlabel('Epoch', fontsize=11)
    plt.ylabel('Loss', fontsize=11)
    plt.title(f'{title_prefix}Model Loss', fontsize=12, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_history(history, 'Feature Extraction - ')

print("📊 Quan sát:")
print("   ✅ Validation accuracy cao ngay từ đầu (transfer learning!)")
print("   ✅ Converge nhanh (chỉ train classifier)")
print("   ✅ Ít overfit (frozen base model)")

In [ ]:
# Evaluate
print("📊 FEATURE EXTRACTION RESULTS:")
print("=" * 60)

train_loss, train_acc = model.evaluate(train_generator, verbose=0)
val_loss, val_acc = model.evaluate(val_generator, verbose=0)

print(f"Training Accuracy:   {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
print("=" * 60)

print("\n✅ Kết quả tốt với chỉ 2000 ảnh training!")
print("   Nhờ pre-trained weights từ ImageNet (1.4M ảnh)")

## 1.6 Fine-tuning

### Chiến lược 2: Fine-tuning

**Fine-tuning** = Unfreeze một số layers cuối và train lại với learning rate nhỏ

### Khi nào dùng Fine-tuning?

✅ **Dùng khi:**
- Feature Extraction chưa đủ tốt
- Có đủ dữ liệu (>5000 images)
- Dataset khác biệt với ImageNet

### Quy trình Fine-tuning

1. Train với Feature Extraction trước
2. Unfreeze một số layers cuối
3. Train lại với learning rate RẤT NHỎ (0.0001)

In [ ]:
# Unfreeze base model
base_model.trainable = True

print(f"📦 Base model có {len(base_model.layers)} layers")
print("\n🔓 Unfreeze các layers cuối...")

# Freeze các layers đầu (100 layers đầu)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Unfreeze các layers cuối
for layer in base_model.layers[100:]:
    layer.trainable = True

# Đếm trainable layers
trainable_layers = sum([layer.trainable for layer in base_model.layers])
print(f"\n✅ Trainable layers: {trainable_layers}/{len(base_model.layers)}")

# Đếm parameters
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
total_params = model.count_params()

print("\n📊 PARAMETERS AFTER UNFREEZING:")
print("=" * 60)
print(f"Total parameters:     {total_params:>15,}")
print(f"Trainable parameters: {trainable_params:>15,}  ({trainable_params/total_params*100:.2f}%)")
print("=" * 60)

print("\n💡 Chú ý:")
print("   - Unfreeze ~50% layers cuối")
print("   - Giữ frozen các layers đầu (low-level features)")
print("   - Train các layers cuối (high-level features)")

In [ ]:
# Recompile với learning rate RẤT NHỎ
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # 10x nhỏ hơn!
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("✅ Model recompiled for fine-tuning!")
print("   Learning rate: 1e-5 (RẤT NHỎ)")
print("\n⚠️  QUAN TRỌNG:")
print("   - Learning rate phải RẤT NHỎ khi fine-tuning")
print("   - Nếu quá lớn → phá hủy pre-trained weights")
print("   - Khuyến nghị: 10-100x nhỏ hơn training từ đầu")

In [ ]:
# Fine-tuning
print("🚀 Fine-tuning model...\n")

# Callbacks cho fine-tuning
finetune_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_finetuned_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

history_finetune = model.fit(
    train_generator,
    epochs=10,  # Ít epochs hơn
    validation_data=val_generator,
    callbacks=finetune_callbacks,
    verbose=1
)

print("\n✅ Fine-tuning completed!")

In [ ]:
# Visualize fine-tuning results
plot_history(history_finetune, 'Fine-tuning - ')

print("📊 Quan sát:")
print("   ✅ Accuracy tăng thêm sau fine-tuning")
print("   ✅ Model adapt tốt hơn với Cats vs Dogs")
print("   ⚠️  Training chậm hơn (train nhiều layers hơn)")

In [ ]:
# Compare Feature Extraction vs Fine-tuning
print("📊 SO SÁNH KẾT QUẢ:")
print("=" * 70)

# Load best feature extraction model
fe_model = keras.models.load_model('best_feature_extraction_model.keras')
fe_val_loss, fe_val_acc = fe_model.evaluate(val_generator, verbose=0)

# Current fine-tuned model
ft_val_loss, ft_val_acc = model.evaluate(val_generator, verbose=0)

print(f"{'Method':<25} {'Val Accuracy':<15} {'Val Loss':<12}")
print("=" * 70)
print(f"{'Feature Extraction':<25} {fe_val_acc:>10.4f}    {fe_val_loss:>10.4f}")
print(f"{'Fine-tuning':<25} {ft_val_acc:>10.4f}    {ft_val_loss:>10.4f}")
print("=" * 70)

improvement = (ft_val_acc - fe_val_acc) * 100
print(f"\n✅ Improvement: +{improvement:.2f}% accuracy")

print("\n💡 Kết luận:")
print("   - Feature Extraction: Nhanh, đủ tốt cho nhiều bài toán")
print("   - Fine-tuning: Chậm hơn nhưng accuracy cao hơn")
print("   - Khuyến nghị: Thử Feature Extraction trước, nếu chưa đủ tốt → Fine-tuning")

## 1.7 Transfer Learning với ResNet50

### ResNet50

- **Residual Networks**: Skip connections
- 50 layers deep
- 25.6M parameters
- Top-1 accuracy: 75.0% trên ImageNet

### Khi nào dùng ResNet50?

✅ Cần accuracy cao hơn MobileNet
✅ Có đủ GPU memory
✅ Không ưu tiên tốc độ inference

In [ ]:
# Load ResNet50
resnet_base = ResNet50(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze
resnet_base.trainable = False

# Build model
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = resnet_base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

resnet_model = keras.Model(inputs, outputs)

print("✅ ResNet50 Model:")
print(f"   Total layers: {len(resnet_base.layers)}")
print(f"   Total parameters: {resnet_model.count_params():,}")
print(f"   Trainable parameters: {sum([tf.size(w).numpy() for w in resnet_model.trainable_weights]):,}")

In [ ]:
# Compile
resnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train
print("🚀 Training ResNet50 model...\n")

history_resnet = resnet_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[
        EarlyStopping(patience=5, restore_best_weights=True),
        ModelCheckpoint('best_resnet_model.keras', save_best_only=True)
    ],
    verbose=1
)

print("\n✅ ResNet50 training completed!")

In [ ]:
# Compare all models
print("📊 SO SÁNH TẤT CẢ CÁC MODELS:")
print("=" * 80)

# Evaluate all models
mobilenet_val_loss, mobilenet_val_acc = fe_model.evaluate(val_generator, verbose=0)
resnet_val_loss, resnet_val_acc = resnet_model.evaluate(val_generator, verbose=0)

results = [
    ('MobileNetV2 (Feature Extraction)', mobilenet_val_acc, mobilenet_val_loss, 3.5),
    ('MobileNetV2 (Fine-tuned)', ft_val_acc, ft_val_loss, 3.5),
    ('ResNet50 (Feature Extraction)', resnet_val_acc, resnet_val_loss, 25.6)
]

print(f"{'Model':<35} {'Val Acc':<12} {'Val Loss':<12} {'Size (M params)'}")
print("=" * 80)

for name, acc, loss, size in results:
    print(f"{name:<35} {acc:>8.4f}    {loss:>10.4f}    {size:>10.1f}M")

print("=" * 80)

print("\n💡 Kết luận:")
print("   - MobileNetV2: Nhỏ, nhanh, phù hợp production")
print("   - ResNet50: Lớn hơn, accuracy có thể cao hơn")
print("   - Fine-tuning: Tăng accuracy nhưng tốn thời gian hơn")
print("\n   Khuyến nghị: MobileNetV2 + Fine-tuning cho hầu hết bài toán!")

## 1.8 Best Practices cho Transfer Learning

### ✅ DO (NÊN LÀM)

#### 1. Data Preparation
```python
# ✅ Normalize theo pre-trained model
# MobileNetV2: [0, 1]
train_datagen = ImageDataGenerator(rescale=1./255)

# ResNet50: mean=[123.68, 116.78, 103.94]
from tensorflow.keras.applications.resnet50 import preprocess_input
```

#### 2. Learning Rate
```python
# ✅ Feature Extraction: Normal LR
optimizer = Adam(learning_rate=0.001)

# ✅ Fine-tuning: RẤT NHỎ
optimizer = Adam(learning_rate=1e-5)
```

#### 3. Layer Freezing
```python
# ✅ Freeze base model khi Feature Extraction
base_model.trainable = False

# ✅ Unfreeze từ từ khi Fine-tuning
for layer in base_model.layers[:100]:
    layer.trainable = False
```

#### 4. Training Strategy
```python
# ✅ Step 1: Feature Extraction (epochs=10-20)
# ✅ Step 2: Fine-tuning (epochs=10)
```

### ❌ DON'T (KHÔNG NÊN)

#### 1. Learning Rate quá lớn khi Fine-tuning
```python
# ❌ WRONG: Phá hủy pre-trained weights
optimizer = Adam(learning_rate=0.01)  # Quá lớn!

# ✅ CORRECT
optimizer = Adam(learning_rate=1e-5)
```

#### 2. Unfreeze tất cả layers ngay từ đầu
```python
# ❌ WRONG
base_model.trainable = True  # Ngay từ đầu!

# ✅ CORRECT: Feature Extraction trước
base_model.trainable = False
# ... train ...
# Sau đó mới unfreeze
```

#### 3. Quên normalize data
```python
# ❌ WRONG: Quên rescale
train_datagen = ImageDataGenerator()  # Pixel values [0, 255]

# ✅ CORRECT
train_datagen = ImageDataGenerator(rescale=1./255)
```

#### 4. Không dùng data augmentation
```python
# ❌ WRONG: Chỉ rescale
train_datagen = ImageDataGenerator(rescale=1./255)

# ✅ CORRECT: Thêm augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    zoom_range=0.2
)
```

### 🎯 Workflow Chuẩn

```python
# STEP 1: Feature Extraction
base_model.trainable = False
model.compile(optimizer=Adam(lr=0.001))
model.fit(epochs=10)

# STEP 2: Fine-tuning (optional)
base_model.trainable = True
# Freeze layers đầu
for layer in base_model.layers[:100]:
    layer.trainable = False
model.compile(optimizer=Adam(lr=1e-5))  # LR nhỏ!
model.fit(epochs=10)
```

---

# PHẦN 2: MIXED PRECISION TRAINING

## 2.1 Mixed Precision là gì?

### Định nghĩa

**Mixed Precision** = Training model với 2 độ chính xác số học khác nhau:
- **Float16 (FP16)**: 16-bit floating point
- **Float32 (FP32)**: 32-bit floating point (default)

### Tại sao dùng Mixed Precision?

| Lợi ích | Giải thích |
|---------|------------|
| 🚀 **Nhanh hơn** | FP16 tính toán nhanh hơn 2-3x (trên GPU hiện đại) |
| 💾 **Ít memory** | FP16 chỉ dùng 50% memory so với FP32 |
| 📊 **Batch size lớn hơn** | Memory tiết kiệm → batch size lớn hơn |
| ✅ **Không giảm accuracy** | Kết quả tương đương FP32 |

### Khi nào dùng Mixed Precision?

✅ **NÊN DÙNG khi:**
- Có GPU hỗ trợ Tensor Cores (V100, A100, RTX 20xx+)
- Model lớn, tốn memory
- Muốn tăng tốc training
- Production environment

❌ **KHÔNG CẦN khi:**
- Chỉ có CPU
- GPU cũ (không hỗ trợ FP16)
- Model nhỏ, train nhanh rồi

### Cách hoạt động

```
Forward pass:  FP16 (nhanh, ít memory)
     ↓
Compute loss:  FP16
     ↓
Backward pass: FP16 (tính gradient)
     ↓
Update weights: FP32 (chính xác)
```

**Loss Scaling:** Nhân gradient với một số lớn để tránh underflow trong FP16

## 2.2 Kiểm tra GPU hỗ trợ Mixed Precision

In [ ]:
# Kiểm tra GPU
gpus = tf.config.list_physical_devices('GPU')

print("🖥️  GPU INFO:")
print("=" * 60)

if len(gpus) == 0:
    print("❌ No GPU found")
    print("   Mixed Precision chỉ hoạt động trên GPU")
    print("   Bạn có thể chạy code này nhưng không thấy speedup")
else:
    print(f"✅ Found {len(gpus)} GPU(s):")
    for i, gpu in enumerate(gpus):
        print(f"   GPU {i}: {gpu.name}")
    
    # Kiểm tra compute capability
    print("\n📊 Mixed Precision Support:")
    print("   GPU với Tensor Cores (compute capability >= 7.0):")
    print("   - V100, A100, RTX 20xx/30xx/40xx: ✅ Full support")
    print("   - GTX 16xx, older: ⚠️  Limited support")

print("=" * 60)

## 2.3 Cấu hình Mixed Precision

### Cách bật Mixed Precision trong TensorFlow 2.x

Rất đơn giản - chỉ cần 1 dòng code:

In [ ]:
# Bật Mixed Precision
from tensorflow.keras import mixed_precision

# Set policy
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print("✅ Mixed Precision enabled!")
print(f"   Compute dtype: {policy.compute_dtype}")
print(f"   Variable dtype: {policy.variable_dtype}")

# Giải thích:
print("\n📚 Giải thích:")
print("   - compute_dtype: float16 → Forward/backward pass dùng FP16")
print("   - variable_dtype: float32 → Weights vẫn lưu ở FP32")
print("   → Nhanh hơn nhưng không mất độ chính xác!")

In [ ]:
# Kiểm tra policy hiện tại
current_policy = mixed_precision.global_policy()
print(f"Current policy: {current_policy}")
print(f"Compute dtype: {current_policy.compute_dtype}")
print(f"Variable dtype: {current_policy.variable_dtype}")

## 2.4 Training với Mixed Precision

### Build model với Mixed Precision

⚠️ **LƯU Ý:** Output layer phải dùng float32 (không phải float16)

In [ ]:
# Build model với Mixed Precision
def create_model_mixed_precision():
    """Create model with mixed precision"""
    # Load base model
    base_model = MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False
    
    # Build model
    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1)(x)  # No activation yet
    
    # ⚠️ QUAN TRỌNG: Output layer PHẢI ở float32
    x = layers.Activation('sigmoid', dtype='float32')(x)
    
    model = keras.Model(inputs, x)
    return model

# Create model
model_mp = create_model_mixed_precision()

print("✅ Mixed Precision Model created!")
print("\n📚 Lưu ý:")
print("   - layers.Activation('sigmoid', dtype='float32')")
print("   - Output layer PHẢI ở float32 để stable")
print("   - Không để output layer ở float16 → numerical instability!")

In [ ]:
# Compile model
# ⚠️ QUAN TRỌNG: Optimizer cần được wrap với LossScaleOptimizer

optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Wrap optimizer với LossScaleOptimizer
optimizer = mixed_precision.LossScaleOptimizer(optimizer)

model_mp.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("✅ Model compiled with Mixed Precision!")
print("\n📚 LossScaleOptimizer:")
print("   - Tự động scale loss để tránh gradient underflow")
print("   - Dynamically điều chỉnh loss scale")
print("   - Giúp training stable với FP16")

## 2.5 So sánh Performance: FP32 vs Mixed Precision

Chúng ta sẽ train 2 models và so sánh:
1. **FP32 (baseline)**: Training thông thường
2. **Mixed Precision**: Training với FP16+FP32

In [ ]:
# Tắt Mixed Precision để train baseline
mixed_precision.set_global_policy('float32')

# Create FP32 model
base_model_fp32 = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model_fp32.trainable = False

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model_fp32(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model_fp32 = keras.Model(inputs, outputs)

# Compile
model_fp32.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("✅ FP32 baseline model created!")

In [ ]:
# Train FP32 model
print("🚀 Training FP32 model...\n")

start_time_fp32 = time.time()

history_fp32 = model_fp32.fit(
    train_generator,
    epochs=5,  # Ít epochs để demo
    validation_data=val_generator,
    verbose=1
)

training_time_fp32 = time.time() - start_time_fp32

print(f"\n✅ FP32 training completed in {training_time_fp32:.2f}s")

In [ ]:
# Bật lại Mixed Precision
mixed_precision.set_global_policy('mixed_float16')

# Train Mixed Precision model
print("🚀 Training Mixed Precision model...\n")

start_time_mp = time.time()

history_mp = model_mp.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
    verbose=1
)

training_time_mp = time.time() - start_time_mp

print(f"\n✅ Mixed Precision training completed in {training_time_mp:.2f}s")

In [ ]:
# Compare results
print("📊 PERFORMANCE COMPARISON:")
print("=" * 70)

# Training time
speedup = training_time_fp32 / training_time_mp
time_saved = training_time_fp32 - training_time_mp

print("TRAINING TIME:")
print(f"  FP32:           {training_time_fp32:>8.2f}s")
print(f"  Mixed Precision:{training_time_mp:>8.2f}s")
print(f"  Speedup:        {speedup:>8.2f}x")
print(f"  Time saved:     {time_saved:>8.2f}s ({time_saved/training_time_fp32*100:.1f}%)")
print()

# Final accuracy
fp32_val_acc = history_fp32.history['val_accuracy'][-1]
mp_val_acc = history_mp.history['val_accuracy'][-1]

print("VALIDATION ACCURACY:")
print(f"  FP32:           {fp32_val_acc:>8.4f}")
print(f"  Mixed Precision:{mp_val_acc:>8.4f}")
print(f"  Difference:     {abs(fp32_val_acc - mp_val_acc):>8.4f}")
print("=" * 70)

print("\n✅ Kết luận:")
if speedup > 1.3:
    print(f"   🚀 Mixed Precision nhanh hơn {speedup:.2f}x!")
    print(f"   💾 Tiết kiệm {time_saved:.1f}s mỗi 5 epochs")
else:
    print(f"   ⚠️  Speedup không rõ ràng (có thể do GPU không hỗ trợ tốt)")

if abs(fp32_val_acc - mp_val_acc) < 0.01:
    print(f"   ✅ Accuracy tương đương nhau")
else:
    print(f"   ⚠️  Accuracy khác biệt đáng kể")

print("\n💡 Lưu ý:")
print("   - Speedup phụ thuộc vào GPU (V100/A100 tốt nhất)")
print("   - Với CPU hoặc GPU cũ, không thấy improvement")
print("   - Model càng lớn, speedup càng rõ ràng")

## 2.6 Best Practices cho Mixed Precision

### ✅ DO (NÊN LÀM)

#### 1. Bật Mixed Precision ngay từ đầu
```python
# ✅ Đặt ở đầu code
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
```

#### 2. Output layer dùng float32
```python
# ✅ CORRECT
x = layers.Dense(1)(x)
x = layers.Activation('sigmoid', dtype='float32')(x)

# Hoặc
outputs = layers.Dense(1, activation='sigmoid', dtype='float32')(x)
```

#### 3. Wrap optimizer
```python
# ✅ CORRECT
optimizer = keras.optimizers.Adam()
optimizer = mixed_precision.LossScaleOptimizer(optimizer)
```

#### 4. Kiểm tra GPU support
```python
# ✅ Check trước khi dùng
gpus = tf.config.list_physical_devices('GPU')
if len(gpus) > 0:
    mixed_precision.set_global_policy('mixed_float16')
```

### ❌ DON'T (KHÔNG NÊN)

#### 1. Quên wrap optimizer
```python
# ❌ WRONG: Không wrap optimizer
optimizer = keras.optimizers.Adam()
model.compile(optimizer=optimizer)  # Thiếu LossScaleOptimizer!

# ✅ CORRECT
optimizer = mixed_precision.LossScaleOptimizer(optimizer)
```

#### 2. Output layer ở float16
```python
# ❌ WRONG
outputs = layers.Dense(1, activation='sigmoid')(x)  # dtype=float16

# ✅ CORRECT
outputs = layers.Dense(1, activation='sigmoid', dtype='float32')(x)
```

#### 3. Dùng Mixed Precision trên CPU
```python
# ❌ WRONG: Waste time checking
mixed_precision.set_global_policy('mixed_float16')  # Trên CPU!

# ✅ CORRECT: Check GPU first
if len(tf.config.list_physical_devices('GPU')) > 0:
    mixed_precision.set_global_policy('mixed_float16')
```

### 🎯 Checklist

- [ ] Có GPU hỗ trợ Tensor Cores?
- [ ] Đã set policy ở đầu code?
- [ ] Output layer dùng dtype='float32'?
- [ ] Optimizer được wrap với LossScaleOptimizer?
- [ ] Đã test accuracy không bị ảnh hưởng?

### 📊 Khi nào thấy speedup rõ ràng?

| Điều kiện | Speedup |
|-----------|----------|
| V100/A100 GPU | 2-3x |
| RTX 30xx | 1.5-2x |
| RTX 20xx | 1.2-1.5x |
| GTX 16xx/CPU | ~1x (không có) |

Model càng lớn, speedup càng rõ ràng!

---

# 🎓 Tổng kết FILE 3-A

## ✅ Những gì đã học

### 1. Transfer Learning
- **Khái niệm**: Sử dụng pre-trained models
- **Chiến lược**:
  - Feature Extraction (freeze base, train classifier)
  - Fine-tuning (unfreeze một số layers, train với LR nhỏ)
- **Pre-trained models**: MobileNetV2, ResNet50
- **Best practices**: LR nhỏ, freeze từ từ, data augmentation

### 2. Mixed Precision Training
- **Khái niệm**: Training với FP16 + FP32
- **Lợi ích**: 2-3x nhanh hơn, ít memory hơn
- **Cách dùng**: 1 dòng code để bật
- **Best practices**: Output layer float32, wrap optimizer

## 🚀 Key Takeaways

1. **Transfer Learning** là must-have cho Computer Vision
2. **Feature Extraction** → nhanh, đủ tốt cho nhiều bài toán
3. **Fine-tuning** → chậm hơn nhưng accuracy cao hơn
4. **Mixed Precision** → free speedup nếu có GPU tốt
5. **MobileNetV2** → best choice cho production

## 📝 Next Steps

File tiếp theo (**3-B**) sẽ học:
- Clean ML Pipeline
- Reproducibility
- Model Evaluation & Metrics

---

**Chúc mừng bạn đã hoàn thành FILE 3-A! 🎉**